In [1]:
import re
import glob
import xlwings as xw
import math
from pathlib import Path
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as mclr
import plotly.express as px
import statsmodels.api as sm
import pylab as py
import os, cv2, glob, tempfile

from scipy import stats
from scipy.stats import pearsonr

import sklearn
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras import utils as np_utils
from keras import models
import keras_tuner as kt
from scipy import signal

import config as config
print(config.__path__)
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras import backend as K

import argparse
# from sklearn.utils import class_weight
print(tf.__version__)
print(keras.__version__)

import shutil
import xlwings as xw

%matplotlib inline
mpl.use("Agg")

['C:\\Users\\user\\anaconda3\\lib\\site-packages\\config']
2.12.0-dev20221107
2.12.0


In [2]:
import warnings
warnings.filterwarnings("ignore")

# 1. Data Preparation

In [3]:
## input FNA set ## 88701 x 20 df
df_ROI_FNA = pd.read_csv(r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_NonIn\postPredict_tPRfNRfDR_NonInFNA_all.csv").drop(columns=["Column1"]).reset_index().drop(columns=["index"])

In [4]:
df_ROI_FNA

,pixel_id,869.7241,909.5496,269.2486,736.6464,535.4734,559.4732,609.5114,682.5922,846.5858,...,type,predicted0n1,p(0),p(1),CM,TPR,FNR,FDR,FPR,TNR
0,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,0.0,-0.003106,-0.195055,0.136741,0.0,0.0,-0.210587,-0.302962,-0.086156,...,0,1,0.0,1.0,FP,0.947195,0.052805,0.487877,0.208711,0.791289
1,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,0.0,-0.333586,0.037120,0.452486,0.0,0.0,-0.210587,-0.302962,-0.086156,...,0,1,0.0,1.0,FP,0.947195,0.052805,0.487877,0.208711,0.791289
2,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,0.0,-0.333586,0.069549,0.402889,0.0,0.0,-0.210587,-0.302962,-0.086156,...,0,1,0.0,1.0,FP,0.947195,0.052805,0.487877,0.208711,0.791289
3,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,0.0,0.124869,0.723098,-0.138512,0.0,0.0,-0.210587,-0.302962,0.913844,...,0,1,0.0,1.0,FP,0.947195,0.052805,0.487877,0.208711,0.791289
4,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,0.0,-0.333586,0.320580,0.433779,0.0,0.0,-0.210587,-0.302962,-0.086156,...,0,1,0.0,1.0,FP,0.947195,0.052805,0.487877,0.208711,0.791289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54837,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_8...,0.0,0.325757,-0.195055,0.295753,0.0,0.0,-0.210587,0.185893,-0.086156,...,1,0,1.0,0.0,FN,1.000000,0.000000,0.812151,1.000000,0.000000
54838,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_9...,0.0,0.245874,0.040356,0.261823,0.0,0.0,-0.210587,0.012085,-0.086156,...,1,0,1.0,0.0,FN,1.000000,0.000000,0.812151,1.000000,0.000000
54839,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_9...,0.0,0.267508,-0.195055,0.216680,0.0,0.0,-0.210587,0.054558,-0.086156,...,1,0,1.0,0.0,FN,1.000000,0.000000,0.812151,1.000000,0.000000
54840,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_9...,0.0,0.177708,-0.195055,0.208274,0.0,0.0,-0.210587,0.206663,-0.086156,...,1,0,1.0,0.0,FN,1.000000,0.000000,0.812151,1.000000,0.000000


In [5]:
df_ROI_FNA["Sample"] = 0
df_ROI_FNA["Row"] = 0
df_ROI_FNA["Scan"] = 0
df_ROI_FNA_Sample = []
df_ROI_FNA_Row = []
df_ROI_FNA_Scan = []
for i in range(len(df_ROI_FNA["pixel_id"])):
    df_ROI_FNA_Sample.append("_".join(str(df_ROI_FNA["pixel_id"][i]).split("_")[:-2]))
    df_ROI_FNA_Row.append(int(str(df_ROI_FNA["pixel_id"][i]).split("_")[-1]))
    df_ROI_FNA_Scan.append(int(str(df_ROI_FNA["pixel_id"][i]).split("_")[-2]))
df_ROI_FNA["Sample"] = df_ROI_FNA_Sample
df_ROI_FNA["Row"] = df_ROI_FNA_Row
df_ROI_FNA["Scan"] = df_ROI_FNA_Scan
pop_column = df_ROI_FNA.pop('Scan')
df_ROI_FNA.insert(1, 'Scan', pop_column)
pop_column = df_ROI_FNA.pop('Row')
df_ROI_FNA.insert(1, 'Row', pop_column)
pop_column = df_ROI_FNA.pop('Sample')
df_ROI_FNA.insert(1, 'Sample', pop_column)

In [6]:
df_ROI_FNA = df_ROI_FNA.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])

In [7]:
df_ROI_FNA

,pixel_id,Sample,Row,Scan,869.7241,909.5496,269.2486,736.6464,535.4734,559.4732,...,type,predicted0n1,p(0),p(1),CM,TPR,FNR,FDR,FPR,TNR
0,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,4,0.0,-0.003106,-0.195055,0.136741,0.0,0.0,...,0,1,0.0,1.0,FP,0.947195,0.052805,0.487877,0.208711,0.791289
1,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,48,33,0.0,-0.333586,0.161001,0.218393,0.0,0.0,...,0,0,1.0,0.0,TN,1.000000,0.000000,0.812151,1.000000,0.000000
2,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,48,32,0.0,-0.333586,0.125459,-0.138512,0.0,0.0,...,0,0,1.0,0.0,TN,1.000000,0.000000,0.812151,1.000000,0.000000
3,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,48,28,0.0,-0.333586,0.122080,-0.138512,0.0,0.0,...,0,0,1.0,0.0,TN,1.000000,0.000000,0.812151,1.000000,0.000000
4,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,48,26,0.0,-0.333586,0.115386,-0.138512,0.0,0.0,...,0,0,1.0,0.0,TN,1.000000,0.000000,0.812151,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54837,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,79,110,0.0,0.176855,-0.195055,0.287703,0.0,0.0,...,1,1,0.0,1.0,TP,0.947195,0.052805,0.487877,0.208711,0.791289
54838,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,79,109,0.0,0.240096,-0.195055,0.237562,0.0,0.0,...,1,1,0.0,1.0,TP,0.947195,0.052805,0.487877,0.208711,0.791289
54839,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,79,108,0.0,0.236392,-0.195055,0.362067,0.0,0.0,...,1,1,0.0,1.0,TP,0.947195,0.052805,0.487877,0.208711,0.791289
54840,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,79,115,0.0,0.241705,0.051219,0.388072,0.0,0.0,...,1,1,0.0,1.0,TP,0.947195,0.052805,0.487877,0.208711,0.791289


In [8]:
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])


In [9]:
sample_ROI_list = [HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_ROI]

In [11]:
name_list = ["HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_ROI"]

In [12]:
sample_ROI_list_FNA = [HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI, 
                       HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_ROI]

In [13]:
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI

,pixel_id,Sample,Row,Scan,869.7241,909.5496,269.2486,736.6464,535.4734,559.4732,...,type,predicted0n1,p(0),p(1),CM,TPR,FNR,FDR,FPR,TNR
0,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,4,0.0,-0.003106,-0.195055,0.136741,0.0,0.0,...,0,1,0.0,1.0,FP,0.947195,0.052805,0.487877,0.208711,0.791289
1,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,5,0.0,-0.333586,0.037120,0.452486,0.0,0.0,...,0,1,0.0,1.0,FP,0.947195,0.052805,0.487877,0.208711,0.791289
2,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,6,0.0,-0.333586,0.069549,0.402889,0.0,0.0,...,0,1,0.0,1.0,FP,0.947195,0.052805,0.487877,0.208711,0.791289
3,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,194,0.0,-0.333586,0.631461,-0.138512,0.0,0.0,...,0,0,1.0,0.0,TN,1.000000,0.000000,0.812151,1.000000,0.000000
4,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,195,0.0,-0.333586,0.571410,-0.138512,0.0,0.0,...,0,0,1.0,0.0,TN,1.000000,0.000000,0.812151,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14820,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,117,144,0.0,0.038599,0.082165,-0.138512,0.0,0.0,...,0,0,1.0,0.0,TN,1.000000,0.000000,0.812151,1.000000,0.000000
14821,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,117,145,0.0,-0.333586,0.078853,-0.138512,0.0,0.0,...,0,0,1.0,0.0,TN,1.000000,0.000000,0.812151,1.000000,0.000000
14822,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,117,146,0.0,-0.333586,0.065736,-0.138512,0.0,0.0,...,0,0,1.0,0.0,TN,1.000000,0.000000,0.812151,1.000000,0.000000
14823,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,118,177,0.0,-0.333586,0.450780,0.220835,0.0,0.0,...,0,0,1.0,0.0,TN,1.000000,0.000000,0.812151,1.000000,0.000000


In [14]:
HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_ROI

,pixel_id,Sample,Row,Scan,869.7241,909.5496,269.2486,736.6464,535.4734,559.4732,...,type,predicted0n1,p(0),p(1),CM,TPR,FNR,FDR,FPR,TNR
0,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_6...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,10,69,0.0,0.257076,-0.195055,0.336380,0.0,0.0,...,1,1,0.0,1.0,TP,0.947195,0.052805,0.487877,0.208711,0.791289
1,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_7...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,11,70,0.0,0.099667,-0.195055,0.394843,0.0,0.0,...,1,1,0.0,1.0,TP,0.947195,0.052805,0.487877,0.208711,0.791289
2,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_6...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,12,69,0.0,0.186832,-0.195055,0.410389,0.0,0.0,...,1,1,0.0,1.0,TP,0.947195,0.052805,0.487877,0.208711,0.791289
3,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_7...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,12,77,0.0,0.125968,-0.195055,0.495215,0.0,0.0,...,1,1,0.0,1.0,TP,0.947195,0.052805,0.487877,0.208711,0.791289
4,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_7...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,12,78,0.0,0.183731,0.002845,0.462439,0.0,0.0,...,1,1,0.0,1.0,TP,0.947195,0.052805,0.487877,0.208711,0.791289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10297,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,102,138,0.0,0.371432,-0.195055,0.296178,0.0,0.0,...,1,1,0.0,1.0,TP,0.947195,0.052805,0.487877,0.208711,0.791289
10298,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,102,139,0.0,0.303087,0.003497,0.272593,0.0,0.0,...,1,1,0.0,1.0,TP,0.947195,0.052805,0.487877,0.208711,0.791289
10299,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,102,140,0.0,0.378887,0.038813,0.320087,0.0,0.0,...,1,1,0.0,1.0,TP,0.947195,0.052805,0.487877,0.208711,0.791289
10300,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,102,141,0.0,0.403062,-0.195055,0.329216,0.0,0.0,...,1,1,0.0,1.0,TP,0.947195,0.052805,0.487877,0.208711,0.791289


# 2. Prediction (made)

## deploy models for training set ##
model_xgb = jl.load(r"H:\3_output_raMSIn\modelXGBoost.joblib")
#
## deploy model ##
predicted0n1_train = model_xgb.predict(X_train)
trainDEFSDf["predicted0n1"] = predicted0n1_train
#
## save ##, ouputing trainSet df
savePath = r"H:\3_output_raMSIn\df_train24_std_0n1.csv"
trainDEFSDf.to_csv(savePath)

pTP_train = model_xgb.predict_proba(X_train)
trainDEFSDf["p(0)"] = list(pd.DataFrame(pTP_train)[0])
trainDEFSDf["p(1)"] = list(pd.DataFrame(pTP_train)[1])
#
## save ##, ouputing trainSet df
savePath = r"H:\3_output_raMSIn\df_train24_std_0n1_pTP.csv"
trainDEFSDf.to_csv(savePath)

In [15]:
( list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI["p(1)"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI["p(0)"]).count(1) + list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI["p(1)"]).count(1)) ) *100, "%"

(32.44519392917369, '%')

In [16]:
( list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI["p(1)"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI["p(0)"]).count(1) + list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI["p(1)"]).count(1)) ) *100, "%"

(11.062711747217037, '%')

In [17]:
( list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI["p(1)"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI["p(0)"]).count(1) + list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI["p(1)"]).count(1)) ) *100, "%"

(18.92210857592447, '%')

In [18]:
( list(HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_ROI["p(1)"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_ROI["p(0)"]).count(1) + list(HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_ROI["p(1)"]).count(1)) ) *100, "%"

(94.71947194719472, '%')

# 3. EDA

In [20]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

In [21]:
import matplotlib.pyplot as plt
import matplotlib.colors
import matplotlib.cm
import numpy as np
from matplotlib.colors import ListedColormap
cmap = cm.binary

def rgb_white2alpha(rgb, ensure_increasing=False):

    # The most transparent alpha we can use is given by the min of RGB
    # Convert it from saturation to opacity
    alpha = 1. - np.min(rgb, axis=1)
    if ensure_increasing:
        # Let's also ensure the alpha value is monotonically increasing
        a_max = alpha[0]
        for i, a in enumerate(alpha):
            alpha[i] = a_max = np.maximum(a, a_max)
    alpha = np.expand_dims(alpha, -1)
    # Rescale colors to discount the white that will show through from transparency
    rgb = (rgb + alpha - 1) / alpha
    # Concatenate our alpha channel
    return np.concatenate((rgb, alpha), axis=1)
    

def cmap_white2alpha(name, ensure_increasing=False, register=True):

    # Fetch the cmap callable
    cmap = plt.get_cmap(name)
    # Get the colors out from the colormap LUT
    rgb = cmap(np.arange(cmap.N))[:, :3]  # N-by-3
    # Convert white to alpha
    rgba = rgb_white2alpha(rgb, ensure_increasing=ensure_increasing)
    # Create a new Colormap object
    cmap_alpha = matplotlib.colors.ListedColormap(rgba, name=name + "_alpha")
    if register:
        matplotlib.cm.register_cmap(name=name + "_alpha", cmap=cmap_alpha)
    return cmap_alpha

In [36]:
# Get our new version of the colormap with white2alpha
cmap_w2a = cmap_white2alpha("Reds")

In [37]:
def genAROItif(sample_ROI, sample_, name, mz, vmax_=1):

    df_heatmap = sample_ROI[["Row", "Scan", mz]]
    df_heatmap2 = pd.DataFrame(np.array([[0, 0, 0]]), columns=['Row', 'Scan', mz]).reset_index().drop(columns = ["index"])
    df_HM = df_heatmap2.loc[df_heatmap["Row"] == 0].loc[df_heatmap2["Scan"] == 0]
    Heat_Row = sample_["Row"]
    Heat_Scan = sample_["Scan"]


    #hi = 0
    for y in range(len(set(Heat_Row))):
        for x in range(1, len(set(Heat_Scan))+1):

            df_in = df_heatmap.loc[df_heatmap["Row"] == (y+1)].loc[df_heatmap["Scan"] == x]
            if len(df_in) == 1:
                df_HM = pd.concat([df_HM, df_in], join="inner", ignore_index=True)
                df_HM = df_HM.sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns = ["index"])
                #hi += 1


            elif len(df_in) == 0:
                df_heatmap2["Row"] = y+1
                df_heatmap2["Scan"] = x
                df_HM = pd.concat([df_HM, df_heatmap2], join="inner", ignore_index=True)
                df_HM = df_HM.sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns = ["index"])
                #hi += 1


    #print(name, mz, hi, len(df_HM))
    if "index" in df_HM.columns:
        df_HM = df_HM.drop(columns = ["index", "Row", "Scan"])
    else:
        df_HM = df_HM.drop(columns = ["Row", "Scan"])

    df_HM = df_HM.T
    New_y = []
    for y in range(1, len(set(Heat_Row))+1):
        New_y.append(str(y))
    New_x = []
    for x in range(1, len(set(Heat_Scan))+1):
        New_x.append(str(x))

    arr = df_HM.values.copy()
    arr.resize(len(set(Heat_Row)), len(set(Heat_Scan)))

    df_HM = pd.DataFrame(arr)
    df_HM.index = New_y
    df_HM.columns = New_x

    for i in list(df_HM.columns):
        df_HM[i] = df_HM[i].astype(float)

    plt.subplots(dpi = 300)
    plt.imshow(df_HM, cmap=cmap_w2a, vmin=0, vmax=1, zorder=1) #cmap_w2a
    plt.xticks([])
    plt.yticks([])
    plt.box(False)
    plt.autoscale(enable=True, tight=True)

    plt.rcParams['figure.figsize'] = [2, 2]

    plt.savefig(f"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_NonIn/{name}_{mz}.tif", bbox_inches = 'tight', pad_inches = 0)

In [24]:
df_raMSI_ML = pd.read_csv(r'H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_NonIn/raMSIn_NonInFNA_noLT.csv').set_index('pixel_id')
df_raMSI_ML.shape

(109685, 631)

In [25]:
raMSI_ML = df_raMSI_ML.reset_index()

In [26]:
raMSI_ML["Sample"] = 0
raMSI_ML["Row"] = 0
raMSI_ML["Scan"] = 0
Sample = []
Row = []
Scan = []
for i in range(len(raMSI_ML["pixel_id"])):
    Sample.append("_".join(str(raMSI_ML["pixel_id"][i]).split("_")[:-2]))
    Row.append(int(str(raMSI_ML["pixel_id"][i]).split("_")[-1]))
    Scan.append(int(str(raMSI_ML["pixel_id"][i]).split("_")[-2]))
raMSI_ML["Sample"] = Sample
raMSI_ML["Row"] = Row
raMSI_ML["Scan"] = Scan

In [27]:
pop_column = raMSI_ML.pop('Scan')
raMSI_ML.insert(1, 'Scan', pop_column)
pop_column = raMSI_ML.pop('Row')
raMSI_ML.insert(1, 'Row', pop_column)
pop_column = raMSI_ML.pop('Sample')
raMSI_ML.insert(1, 'Sample', pop_column)

In [28]:
raMSI_ML = raMSI_ML.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])

In [30]:
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])


In [31]:
sample_list = [HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3]

In [38]:
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime

TOP ="p(1)"
    

counter = 0

for im in range(len(sample_ROI_list)):
    start = datetime.now()

    name = name_list[im]
    sample_ = sample_list[im]
    preview = sample_ROI_list[im][[TOP]]
    for i in preview.columns:
        preview[i] = preview[i].astype(float)

    vmax_list = list(preview.describe().iloc[-1])
    vmax_ = 1
    #print(TOP, name, vmax_)

    genAROItif(sample_ROI=sample_ROI_list[im], sample_=sample_, name=name, mz=TOP, vmax_=vmax_)
    #print(df.shape)
    counter += 1
    end = datetime.now()
    Duration = end - start
    print(Duration)


0:00:58.311276
0:00:56.083327
0:01:00.171655
0:00:35.152136
